In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import display

from PIL import Image
from src.draw_util import draw_x_at, composite_at

In [ ]:
dartboard_filename = 'res/dart_board.png'

In [ ]:
dartboard_image = Image.open(dartboard_filename)
dartboard_image

In [ ]:
width, height = dartboard_image.size
print(width, height)
left = 450
top = 350
right = width - 480
bottom = height
cropped = dartboard_image.crop((left, top, right, bottom))
cropped.save('res/dart_board_cropped.png')
cropped

In [ ]:
cropped_with_anchors = cropped.copy()

top_anchor = (495, 195)
left_anchor = (232, 425)
right_anchor = (750, 442)
bottom_anchor = (487, 643)

cropped_with_anchors = draw_x_at(cropped_with_anchors, top_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, left_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, right_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, bottom_anchor)

display(cropped_with_anchors)

In [ ]:
# draw dart on background board
dart_1 = Image.open('res/darts/dart_1.png')
composite = composite_at(cropped, dart_1, (300, 200))
composite.convert('RGB')

In [ ]:
def dart_position_in_cropped(pasted_at_pos, dart_top_pos):
    return (pasted_at_pos[0] + dart_top_pos[0], pasted_at_pos[1] + dart_top_pos[1])

def dart_position_in_original(cropped_at_pos, pasted_at_pos, dart_top_pos):
    return (cropped_at_pos[0] + pasted_at_pos[0] + dart_top_pos[0], cropped_at_pos[1] + pasted_at_pos[1] + dart_top_pos[1])

In [ ]:
center = (490, 432)
center_x = draw_x_at(cropped, center)
center_x

In [ ]:

###
### Generate Config
###
from src.data_generation import DataGenerationConfig


config: [DataGenerationConfig] = [
    #DataGenerationConfig(5, ({'type': 'random'}, {'type': 'random'}, {'type': 'random'})),
]

# two darts close
"""

for angle in range(0, 360, 45):
    for selection_radius_abs in [40]:
        for radius_relativ in [x / 10 for x in range(0, 11, 2)]:
            config.append(DataGenerationConfig(1, (
                {'type': 'radius', 'radius': radius_relativ, 'selection_radius_abs': selection_radius_abs, 'angle': angle},
                {'type': 'radius', 'radius': radius_relativ, 'selection_radius_abs': selection_radius_abs, 'angle': angle},
                {'type': 'random'}
            )))

"""
# three darts close
config.append(DataGenerationConfig(1, (
    {'type': 'radius', 'radius': 0, 'selection_radius_abs': 20, 'angle': 0},
    {'type': 'radius', 'radius': 0, 'selection_radius_abs': 20, 'angle': 0},
    {'type': 'radius', 'radius': 0, 'selection_radius_abs': 20, 'angle': 0},
)))
for angle in range(0, 360, 45):
    for selection_radius_abs in [30]:
        for radius_relativ in [x / 10 for x in range(3, 11, 3)]:
            config.append(DataGenerationConfig(1, (
                {'type': 'radius', 'radius': radius_relativ, 'selection_radius_abs': selection_radius_abs, 'angle': angle},
                {'type': 'radius', 'radius': radius_relativ, 'selection_radius_abs': selection_radius_abs, 'angle': angle},
                {'type': 'radius', 'radius': radius_relativ, 'selection_radius_abs': selection_radius_abs, 'angle': angle},
            )))
#config

In [ ]:
###
### Dart Config
###
import json

from src.data_generation import DartConfig

dart_config: DartConfig = []

darts_config_file = 'darts.json'
with open(darts_config_file) as f:
    darts = json.load(f)

for id_, details in darts.items():
    print(f"dart id {id_} with {len(details)} images")
    dart_lst = []
    for i in details:
        img = Image.open(i['path'])
        dart_lst.append((img, i['pos']))
    dart_config.append(DartConfig(id_, dart_lst))

In [ ]:
from src.data_generation import DataGenerator, Position, PositionGenerator

center_pos = Position(center[0], center[1])
top_anchor_pos = Position(top_anchor[0], top_anchor[1])
left_anchor_pos = Position(left_anchor[0], left_anchor[1])
right_anchor_pos = Position(right_anchor[0], right_anchor[1])
bottom_anchor_pos = Position(bottom_anchor[0], bottom_anchor[1])

position_generator = PositionGenerator(
    center=center_pos,
    anchor_top=top_anchor_pos,
    anchor_left=left_anchor_pos,
    anchor_right=right_anchor_pos,
    anchor_bottom=bottom_anchor_pos,
)

data_generator = DataGenerator(
    anchor_top=top_anchor_pos,
    anchor_left=left_anchor_pos,
    anchor_right=right_anchor_pos,
    anchor_bottom=bottom_anchor_pos,
    position_generator=position_generator
)

labels = data_generator.generate(config, dart_config, cropped, 'data/refactored')